# ETL Worfklow


---

## Setup


Importing my libraries of choice

In [6]:
import os
import pyspark.sql
import requests

Getting our SQL Server credientials ahead of time

In [3]:
db_user = os.environ.get('DB_USER')
db_password = os.environ.get('DB_PASSWORD')

Creating a spark session 

In [4]:
spark_sesh = pyspark.sql.SparkSession.builder.appName('Credit Card ETL').getOrCreate()

In [5]:
spark_sesh

---

## Extract 

I'm going to extract data from two datasets pertaining to the same bank, which are:


- The Credit Card Dataset
    - cdw_sapp_customer.JSON
    - cdw_sapp_branch.JSON 
    - cdw_sapp_credit.JSON

<br>

- The Bank Loan Application Dataset
    - Loan Application API Endpoint

📝 Notes: <br>
- The customer.JSON file and branch.JSON file contains information about the bank customer and bank branch, respectively. 
- The credit.JSON file contains information about credit card transactions
- The Loan Application is with respect to loans for purchasing homes, and includes information like whether or not the individuals were approved, gender, maritual status, and income.

Extracting the JSON Files with sparksession.read.load()


In [8]:
# JSON Files
branch_df = spark_sesh.read.load('../../data/credit_card_dataset/cdw_sapp_branch.json', format='json')  # 👀 don't forget the to specify the format
credit_df = spark_sesh.read.load('../../data/credit_card_dataset/cdw_sapp_credit.json', format='json')
customer_df = spark_sesh.read.load('../../data/credit_card_dataset/cdw_sapp_custmer.json', format='json')

Defining an API endpoint


In [15]:
LOAN_API_ENDPOINT = "https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json"


Creating a function to create a dataframe from an API Endpoint: <br>
 
The function takes an API Endpoint URL and the current live spark session, <br>
checks to see if the HTTP request is OK and returns a pyspark.sql DataFrame.

In [23]:
# API 

def api_check(api_endpoint: str, spark_session: pyspark.sql.SparkSession) -> pyspark.sql.DataFrame:  # param :type -> output-type
    api = requests.get(LOAN_API_ENDPOINT)
    print(f"HTTP Status Code: {api.status_code}")
    if api.status_code == 200:
        api_df = spark_session.createDataFrame(api.json())
        return api_df



In [24]:
loan_df = api_check(LOAN_API_ENDPOINT, spark_sesh)

HTTP Status Code: 200


Let's check our DataFrames and make sure they are intact and ready for transformation

----